In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib import request
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import re
import sys
import json

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

In [30]:
# """
# 起動1
# java -jar C:\Users\mmi-lab\Documents\selenium\selenium-server-standalone-3.141.59.jar -role hub

# 起動2
# java -Dwebdriver.chrome.driver="C:\Users\mmi-lab\Documents\selenium\chromedriver.exe" -jar C:\Users\mmi-lab\Documents\selenium\selenium-server-standalone-3.141.59.jar -role node -hub http://192.168.1.224:4444

# """

# """
# 使えそう
# ジャンルからフレーズ・例文を探す
# https://gogakuru.com/english/phrase/genre/index.html

# """
print("aa")

aa


In [31]:
import time
from selenium.webdriver.common.by import By
import spacy
import csv
import copy

class Crawler:
    def __init__(self, corpus_path, name) -> None:
        self.query = ""
        self.stop_word = "← ww ・ 「 」 #".split()
        self.nlp =  spacy.load('ja_ginza')

        self.corpus_path = corpus_path
        self.name = name

        self.tweet_all = set()
    
    def connect(self, server=None):
        chrome_options = webdriver.ChromeOptions()
        if server:
            command = "http://{0}:4444/wd/hub".format(server)
            self.driver = webdriver.Remote(
                command_executor="http://{0}:4444/wd/hub".format(server),
                options=chrome_options
            )
            
    def _in(self, arg1, arg2, mode="or"):
        """
            arg1: str or list
            arg2: str
        """
        result = False
        if isinstance(arg1, list):
            if mode == "and":
                for element in arg1:
                    if isinstance(element, str):
                        if element not in arg2:
                            return False
                return True
            # or
            for element in arg1:
                if isinstance(element, str):
                    # print("str")
                    if element in arg2:
                        result = True
                        break
                else:
                    if element == arg2:
                        result = True
                        break
        else:
            if isinstance(arg1, str):
                if arg1 in arg2:
                    result = True
            else:
                if arg1 == arg2:
                    result = True
        return result
        
    def query2formated(self, query):
        splited = query.split()
        return "(" + " OR ".join(splited) + ")" + " -filter:links"
    
    def crawl(self, query, num):
        url = "https://twitter.com/search?f=live&q="
        self.query_list = query.split()
        self.driver.get( url+self.query2formated(query) )
        WebDriverWait(self.driver, 10).until(EC.visibility_of_element_located((By.TAG_NAME, 'article')))
        
        
        tweet_list = self.get_tweets()
        while( len(self.tweet_all) < num ):
            self.get_tweets()
            # self.tweet_all =  self.tweet_all + copy.deepcopy(tweet_list)
            self._scroll()
            # print(len(self.tweet_all))
            time.sleep(3)

            
        print("conplete crawled tweets, the size is : ", len(self.tweet_all))
        # searchbox.send_keys("ポケモン")
        # searchbox.send_keys(Keys.ENTER)

    def _scroll(self):
        articles = self.driver.find_elements_by_tag_name('article')
        last_art = articles[-2]
        actions = ActionChains(self.driver)
        actions.move_to_element(last_art)
        actions.perform()
    
    def extract_info(self, data):
        soup = BeautifulSoup(data, features='lxml')

    def _is_invalid_tweet(self, text):
        # return (self._in(self.query_list, text) and  not self._in(self.stop_word, text) )
        
        if not self._in(self.query_list, text):
            return True
        
        if self._in(self.stop_word, text):
            return True
        
        # 発話が長すぎるのも駄目でしょう
        if len( self.nlp(text) ) >= 15:
            return True
        
        return False
    
    def get_tweets(self):
        html = self.driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        # articles = self.driver.find_elements_by_tag_name('article')
        articles = soup.find_all("article")
        # print("articles : ", len(articles))
        tweet_list = []
        for art in articles:
            # contents = art.find_elements_by_css_selector('.css-901oao.css-16my406.r-1tl8opc r-bcqeeo.r-qvutc0')
            # contents = art.find_elements_by_tag_name('span')
            cell = art.find("div",{'lang': 'ja'})
            try:
                contents = cell.find_all("span")
                text = ""
                for content in contents:
                    text += content.get_text().replace('\n','')


                # しっかりと正規化
                text = clean_text(text)
                if text in self.tweet_all:
                    continue
                # チェック
                if not self._is_invalid_tweet(text):
                    # print(text)
                    # tweet_list.append(text)
                    self.tweet_all.add(text)

                    if len(self.tweet_all) % 30 == 0:
                        print("crawled : ", len(self.tweet_all))
            except:
                print("not japanese")
            # print(contents.get_text())
        # print(tweet_list)
        # return tweet_list

    def save_tweet(self):
        with open(self.corpus_path+self.name, "w") as f:
            writer = csv.writer(f)
            for t in list(self.tweet_all):
                writer.writerow(["0", t])

    def load_tweet(self, name):
        with open(self.corpus_path+name, "r") as f:
            reader = csv.reader(f)
            for r in reader:
                self.tweet_all.add(r[1])
        
        print("success loading from {0}, the size is {1}".format(self.corpus_path+name, len(self.tweet_all)))

            


In [32]:
out_path = "../../corpus/twitter/"
name = "hate2.csv"
crawler = Crawler(out_path, name)
crawler.connect("192.168.1.224")

In [33]:
crawler.load_tweet("hate.csv")

success loading from ../../corpus/twitter/hate.csv, the size is 925


In [39]:

crawler.crawl("馬鹿　バカ　アホ", 3000)

crawled :  1020
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
crawled :  1050
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
crawled :  1080
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
crawled :  1110
not japanese
not japanese
not japanese
crawled :  1140
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
crawled :  1170
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
crawled :  1200
not japanese
not japanese
not japanese
not japanese
not japanese
crawled :  1230
not japanese
not japanese
not japanese
not japanese
not japanese
not japanese
crawled :  1260
crawled :  1290
not japanese
not japanese
not japanese
not japanese
crawled :  1320
not japanese
not j

KeyboardInterrupt: 

In [41]:
crawler.save_tweet()

In [40]:
len(crawler.tweet_all)

1560